In [1]:
import pandas as pd
df=pd.read_csv('crypto_price_prediction_dataset.csv')

In [2]:
df.head()

,O1,H1,L1,C1,V1,O2,H2,L2,C2,V2,...,L48,C48,V48,macd,macd_signal,rsi,sma20,sma50,sma200,target
0,46216.93,46731.39,46208.37,46656.13,1503.33095,46656.14,46949.99,46574.06,46778.14,943.81539,...,47255.38,47286.18,749.48136,-4.998207,9.570417,50.279883,47190.8480,NaN,NaN,9
1,46656.14,46949.99,46574.06,46778.14,943.81539,46778.14,46928.94,46721.96,46811.77,485.16860,...,47050.00,47050.08,734.11794,-16.372949,4.381744,47.182587,47197.8370,NaN,NaN,9
2,46778.14,46928.94,46721.96,46811.77,485.16860,46811.77,46916.63,46760.12,46813.20,562.88971,...,46950.00,47116.71,628.05580,-19.782984,-0.451202,47.608367,47199.6085,47175.0758,NaN,10
3,46811.77,46916.63,46760.12,46813.20,562.88971,46813.21,46887.33,46591.23,46711.05,861.88389,...,46858.44,46999.98,781.70714,-31.541012,-6.669164,44.239066,47193.4770,47181.9528,NaN,9
4,46813.21,46887.33,46591.23,46711.05,861.88389,46711.05,47555.55,46673.94,47192.55,1400.73642,...,46722.39,46793.26,850.43854,-56.884185,-16.712168,43.367195,47174.4675,47182.2552,NaN,8


In [11]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GRU, Dense, Dropout, Concatenate
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# 1. Загрузка данных
data = pd.read_csv('crypto_price_prediction_dataset.csv')

# 2. Подготовка данных
# axis=1 - применение операции двигаясь вбок по стольбцам
X = data.drop('target', axis=1).values  # Все фичи (246 колонок)
y = data['target'].values  # Целевые классы

# Разделение на train/test (80/20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Нормализация данных
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Замена возможных NaN/Inf после нормализации
X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

# 5. Создание модели с защитой от взрыва градиентов
input_sequences = Input(shape=(48, 5))
gru1 = GRU(128, return_sequences=True, kernel_constraint=tf.keras.constraints.MaxNorm(3))(input_sequences)
drop1 = Dropout(0.3)(gru1)
gru2 = GRU(64, kernel_constraint=tf.keras.constraints.MaxNorm(3))(drop1)

input_indicators = Input(shape=(6,))

combined = Concatenate()([gru2, input_indicators])

# Добавляем BatchNormalization и дополнительный Dense слой
norm = tf.keras.layers.BatchNormalization()(combined)
dense = Dense(32, activation='relu')(norm)
output = Dense(13, activation='softmax')(dense)

model = Model(inputs=[input_sequences, input_indicators], outputs=output)

# 6. Компиляция с уменьшенным learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(
    optimizer=optimizer,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# 7. Добавляем EarlyStopping
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

# 8. Обучение с уменьшенным batch_size
history = model.fit(
    [X_train_sequences, X_train_indicators], y_train,
    validation_data=([X_test_sequences, X_test_indicators], y_test),
    epochs=100,  # Увеличиваем эпохи, так как EarlyStopping остановит при необходимости
    batch_size=8,  # Уменьшаем batch size
    callbacks=[checkpoint, early_stop],
    verbose=1
)

# 9. Оценка
test_loss, test_acc = model.evaluate([X_test_sequences, X_test_indicators], y_test)
print(f"\nТочность на тестовых данных: {test_acc:.4f}")

Epoch 1/100


C:\Users\Rodion\AppData\Roaming\Python\Python312\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor_21', 'keras_tensor_25']. Received: the structure of inputs=('*', '*')
  warnings.warn(


3086/3086 ━━━━━━━━━━━━━━━━━━━━ 121s 37ms/step - accuracy: 0.2447 - loss: 2.5505 - val_accuracy: 0.2484 - val_loss: 2.4961
Epoch 2/100
3086/3086 ━━━━━━━━━━━━━━━━━━━━ 113s 37ms/step - accuracy: 0.2484 - loss: 2.4809 - val_accuracy: 0.2484 - val_loss: 2.4469
Epoch 3/100
3086/3086 ━━━━━━━━━━━━━━━━━━━━ 73s 24ms/step - accuracy: 0.2500 - loss: 2.4351 - val_accuracy: 0.2484 - val_loss: 2.4133
Epoch 4/100
3086/3086 ━━━━━━━━━━━━━━━━━━━━ 47s 15ms/step - accuracy: 0.2487 - loss: 2.4044 - val_accuracy: 0.2484 - val_loss: 2.3917
Epoch 5/100
3086/3086 ━━━━━━━━━━━━━━━━━━━━ 49s 16ms/step - accuracy: 0.2479 - loss: 2.3844 - val_accuracy: 0.2484 - val_loss: 2.3788
Epoch 6/100
3086/3086 ━━━━━━━━━━━━━━━━━━━━ 49s 16ms/step - accuracy: 0.2476 - loss: 2.3739 - val_accuracy: 0.2484 - val_loss: 2.3717
Epoch 7/100
3086/3086 ━━━━━━━━━━━━━━━━━━━━ 51s 16ms/step - accuracy: 0.2539 - loss: 2.3602 - val_accuracy: 0.2484 - val_loss: 2.3680
Epoch 8/100
3086/3086 ━━━━━━━━━━━━━━━━━━━━ 56s 18ms/step - accuracy: 0.2524 - l

In [3]:
# упрощенный код (без защиты от взыра градиента)
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GRU, Dense, Dropout, Concatenate
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# 1. Загрузка данных
data = pd.read_csv('crypto_price_prediction_dataset.csv')

# 2. Подготовка данных
X = data.drop('target', axis=1).values  # Все фичи (246 колонок)
y = data['target'].values  # Целевые классы (добавили .values чтобы получить не pandas series, а массим numpy (это нужно для tensorflow))

# Разделение на train/test (80/20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# random_state=42 - семя для генератора случайных чисел. При разных random_state разное разбиение (при random_state=None разбиение каждый раз случайное)

# 3. Нормализация данных
scaler = MinMaxScaler() # масштабирует данные в диапазон [0,1]
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Замена возможных NaN/Inf после нормализации
X_train = np.nan_to_num(X_train) # заменяем NaN на 0, inf на очень большое число, -inf на очень маленькое
X_test = np.nan_to_num(X_test)

# 5. Создание модели (без защиты от взрыва градиентов)
# тут используется Functional API - этим методом создается сложные модели с несколькими входами
input_sequences = Input(shape=(48, 5)) # вход для данных по свечам
gru1 = GRU(128, return_sequences=True)(input_sequences)
drop1 = Dropout(0.3)(gru1)
gru2 = GRU(64)(drop1)

input_indicators = Input(shape=(6,)) # вход для данных по индикаторам

combined = Concatenate()([gru2, input_indicators])

# Добавляем BatchNormalization и дополнительный Dense слой
norm = tf.keras.layers.BatchNormalization()(combined)
dense = Dense(32, activation='relu')(norm)
output = Dense(13, activation='softmax')(dense)

model = Model(inputs=[input_sequences, input_indicators], outputs=output)

# Вход 1: (48, 5) → GRU → GRU → Объединение → Dense → Выход  
# Вход 2: (6,) ────────────────┘  

# 6. Компиляция
# компиляция нужна чтобы сказать модели как обновлять весса, как отслеживать ошбику, какие метрики использовать 
optimizer = tf.keras.optimizers.Adam()
model.compile(
    optimizer=optimizer,
    loss='sparse_categorical_crossentropy', # нужно для обучения
    metrics=['accuracy'] # нужно для понимания человеком
)

# 7. Добавляем EarlyStopping
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', # отслеживаем значение функции потерь на валидационной выборке
    patience=5, # если val_loss не улучшается 5 эпох подряд -> обучение останавлявается
    restore_best_weights=True # после остановки возвращает веса из лучшей эпохи
)

# Создание checkpoint
checkpoint = ModelCheckpoint(
    filepath='best_model.keras',  # Путь для сохранения
    monitor='val_loss',        # Отслеживаем val_loss
    save_best_only=True,       # Сохранять только лучшую модель
    mode='min',               # Чем меньше val_loss, тем лучше
    verbose=1                 # Вывод сообщений
)

# 8. Обучение
X_train_indicators=X_train[:,-6:]
X_train_sequences=X_train[:,:-6]
X_test_sequences=X_test[:, :-6]
X_test_indicators=X_test[:,-6:]
history = model.fit(
    [X_train_sequences, X_train_indicators], y_train,
    validation_data=([X_test_sequences, X_test_indicators], y_test),
    epochs=100,
    batch_size=32,
    callbacks=[checkpoint, early_stop],
    verbose=1
)

# 9. Оценка
test_loss, test_acc = model.evaluate([X_test_sequences, X_test_indicators], y_test) # возвращает значение loss и accuracy
print(f"\nТочность на тестовых данных: {test_acc:.4f}")

Epoch 1/100


ValueError: Layer "functional_2" expects 2 input(s), but it received 1 input tensors. Inputs received: [<tf.Tensor 'data:0' shape=(None, 246) dtype=float32>]